In [1]:
import dspy
import ujson
import random

from dspy.evaluate import SemanticF1
from dspy.utils import download

# Download Data

In [2]:
download(
    "https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl"
)
with open("ragqa_arena_tech_examples.jsonl", "r") as fp:
    data = [ujson.loads(line) for line in fp]

In [3]:
data[0]

{'question': 'why igp is used in mpls?',
 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.",
 'gold_doc_ids': [2822, 2823]}

# Convert Data to DSPy Examples

In [4]:
data = [dspy.Example(**d).with_inputs("question") for d in data]
data[2]

Example({'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}) (input_keys={'question'})

In [5]:
len(data)

2064

# Train/Test Split

In [6]:
random.Random(23).shuffle(data)
train, dev, test = data[:200], data[200:500], data[500:]

# DSPy Setups

In [7]:
lm = dspy.LM(model="ollama_chat/llama3.1", temperature=0.0)
dspy.configure(lm=lm)

cot = dspy.ChainOfThought("question -> response")

# Evaluation in DSPy

In [8]:
metric = SemanticF1(decompositional=True)
pred = cot(**data[2].inputs())
score = metric(data[2], pred)

print(f"Question:\t {data[2].question}")
print(f"Gold Response:\t {data[2].response}")
print(f"Pred Response:\t {pred.response}")
print(f"Semantic F1 Score:\t {score:.2f}")

Question:	 how can i show typing keyboard in record screen
Gold Response:	 One method involves using Screenflow, a software that includes this feature and is priced at $99.  
Additionally, standalone apps like Keycastr and Mouseposé can be used for the same purpose.  
Another alternative is showing the on-screen keyboard, although this results in a different aesthetic.
Pred Response:	 To show a typing keyboard in a recorded screen using OBS Studio:

1. Open OBS Studio and add a new source by clicking the "+" button.
2. Select "Virtual Keyboard" from the list of available sources.
3. Customize the appearance of the virtual keyboard as desired, including its size, color, and font.
4. Position the virtual keyboard within the recording area to match your desired layout.
5. Start recording your screen using OBS Studio.

By following these steps, you can add a typing keyboard to your recorded screen in OBS Studio.
Semantic F1 Score:	 0.86


In [9]:
evaluate = dspy.Evaluate(
    devset=dev, metric=metric, num_threads=4, display_progress=True, display_table=True
)
evaluate(cot)

Average Metric: 169.96 / 299 (56.8%): 100%|█████████▉| 299/300 [17:56<00:02,  2.37s/it]

2024/12/30 02:40:58 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'return only the portion of a line after a matching pattern', 'response': 'The primary tool for text manipulation and pattern replacement is sed, which operates with commands like: `sed -n -e \'s/^.*stalled: //p\'`, which are designed to search and transform strings within text files. \nThe following sed command, utilizing groups and backreferences, swaps the sections before and after the specified match string: `sed -n -e \'s/^\\(.*\\)\\(stalled: \\)\\(.*\\)$/\\3\\2\\1/p\'`. \nAnother tool, grep, like: "grep -o \'stalled.*\', provides the ability to return only the matching parts of text lines through the use of specific flags like -o. \nAdditionally, the cut command: "grep -o \'stalled.*\' | cut -f2- -d" can be employed for extracting certain parts of a text line by using delimiters to specify the fields you want to keep. \nAlternatively, awk command: "awk -F"stalled" \'/stalled/{print $2}\'

Average Metric: 169.96 / 299 (56.8%): 100%|██████████| 300/300 [18:09<00:00,  3.63s/it]

2024/12/30 02:40:58 INFO dspy.evaluate.evaluate: Average Metric: 169.96268669706683 / 300 (56.7%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1,response
0,vtdecoderxpcservice taking cpu,Quit your applications one by one and monitor the process. Common ...,"[2557, 4732, 5014, 5156, 5195, 5631, 5856, 4026]",The high CPU usage by vtdecoderxpcservice is likely due to a resou...,Try restarting the vtdecoderxpcservice to see if it resolves the h...,✔️ [0.353],NaN
1,git auto-complete for *branches* at the command line?,"You should try to update the git version to the latest, and then i...","[6715, 1766, 2902, 6154]",To find the correct command to auto-complete branches in Git at th...,The command for enabling auto-completion is `source /usr/share/git...,,NaN
2,what functionality do marks offer in the el capitan terminal?,"Marks in the Terminal, which began with OS X 10.11 - El Capitan, h...","[1234, 1253]",The question is asking about the functionality of marks in the El ...,"Marks offer several functionalities in the El Capitan terminal, in...",✔️ [0.571],NaN
3,how to convert a heif/heic image to jpeg in el capitan?,Users advise that tifig is an excellent command line tool that wor...,"[7785, 680, 2705, 7201]","To convert a HEIF/HEIC image to JPEG in El Capitan, we can use the...",To convert a HEIF/HEIC image to JPEG in El Capitan: 1. Open Termin...,✔️ [0.667],NaN
4,why should i use a factory class instead of direct object construc...,Using factory classes along with interfaces promotes greater long-...,[4782],Using a factory class instead of direct object construction provid...,"Use a factory class to control and decouple object creation, impro...",✔️ [0.857],NaN
...,...,...,...,...,...,...,...
295,make the menu bar never show while in full screen,Some users report that the menu bar on Mac OS X cannot be hidden o...,"[5370, 4551, 44, 6307]","To make the menu bar never show while in full screen, we can use C...",You can achieve this by adding the following CSS code: ```css @med...,✔️ [1.000],NaN
296,whats a good ssh tunneling client for os x?,"SSHTunnel, a free application for managing SSH tunnels, is still g...","[2186, 2809, 2814, 2817, 526, 5277, 6415, 1635, 1671, 6739, 2868, ...","To answer this question, I'll consider popular SSH tunneling clien...","Based on the options considered, Tunnelblick seems to be a good ch...",✔️ [0.200],NaN
297,delete line in vi,"Use the ""dd"" command, which will remove the line, allowing you to ...","[3122, 3123, 3129, 5910, 4792, 2164]","The command to delete a line in vi is 'd' followed by '$', which m...",'d$',,NaN
298,return only the portion of a line after a matching pattern,NaN,"[5023, 958, 6330]",NaN,NaN,,The primary tool for text manipulation and pattern replacement is ...


56.65